In [3]:
import pandas as pd

In [4]:
t_edu = pd.read_csv("./loc_data/taipei_edu.csv")
t_hospital = pd.read_csv("./loc_data/taipei_hos.csv")
t_trans = pd.read_csv("./loc_data/taipei_trans.csv")
t_other = pd.read_csv("./loc_data/taipei_other.csv")
t_store = pd.read_csv("./loc_data/taipei_store.csv")
t_population = pd.read_csv("./loc_data/taipei_population.csv")

nt_edu = pd.read_csv("./loc_data/new_taipei_edu.csv")
nt_hospital = pd.read_csv("./loc_data/new_taipei_hos.csv")
nt_trans = pd.read_csv("./loc_data/new_taipei_trans.csv")
nt_other = pd.read_csv("./loc_data/new_taipei_other.csv")
nt_store = pd.read_csv("./loc_data/new_taipei_store.csv")
nt_population = pd.read_csv("./loc_data/new_taipei_population.csv")

house = pd.read_csv("./loc_data/house.csv")
pt = pd.read_csv("points_taipei.csv")
pnt = pd.read_csv("points_new_taipei.csv")


C:\Users\cxh83\AppData\Local\Temp/ipykernel_13348/1097734581.py:12: DtypeWarning: Columns (4,6,8,10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  nt_store = pd.read_csv("./loc_data/new_taipei_store.csv")


In [5]:
# 將 store 中的台灣房屋個數 信義房屋個數 永慶房屋個數 有巢氏房屋個數 住商房屋個數加總
t_store['房仲業個數'] = t_store['台灣房屋個數'] + t_store['信義房屋個數'] + t_store['永慶房屋個數'] + t_store['有巢氏房屋個數'] + t_store['住商房屋個數']
nt_store['房仲業個數'] = nt_store['台灣房屋個數'] + nt_store['信義房屋個數'] + nt_store['永慶房屋個數'] + nt_store['有巢氏房屋個數'] + nt_store['住商房屋個數']

In [6]:
# merge on x1,x2,y1,y2
pt_merge = pd.merge(pt, t_edu, on=['x1','x2','y1','y2'], how='left')
pt_merge = pd.merge(pt_merge, t_hospital, on=['x1','x2','y1','y2','index'], how='left')
pt_merge = pd.merge(pt_merge, t_trans, on=['x1','x2','y1','y2','index'], how='left')
pt_merge = pd.merge(pt_merge, t_other, on=['x1','x2','y1','y2','index'], how='left')
pt_merge = pd.merge(pt_merge, t_store, on=['x1','x2','y1','y2'], how='left')
pt_merge = pd.merge(pt_merge, t_population, on=['x1','x2','y1','y2'], how='left')
pt_merge = pd.merge(pt_merge, house, on=['x1','x2','y1','y2'], how='left')
pt_merge


,x1,y1,x2,y2,index,edu1_100_cnt,edu1_500_cnt,edu1_1000_cnt,edu1_3000_cnt,edu2_100_cnt,...,平均每坪單價_2022,平均每坪單價_2021,平均每坪單價_2020,平均每坪單價_2019,交易總價_2024,交易總價_2023,交易總價_2022,交易總價_2021,交易總價_2020,交易總價_2019
0,296103,2777535,296203,2777635,0,0,0,1,30,0,...,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,0.0,0.0,0.0,0.0,0.0,0.0
1,296103,2777635,296203,2777735,1,0,0,1,26,0,...,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,0.0,0.0,0.0,0.0,0.0,0.0
2,296103,2777735,296203,2777835,2,0,0,0,27,0,...,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,0.0,0.0,0.0,0.0,0.0,0.0
3,296103,2777835,296203,2777935,3,0,0,0,27,0,...,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,0.0,0.0,0.0,0.0,0.0,0.0
4,296103,2777935,296203,2778035,4,0,0,0,27,0,...,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27656,317103,2768935,317203,2769035,27656,0,0,0,5,0,...,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,0.0,0.0,0.0,0.0,0.0,0.0
27657,317103,2769035,317203,2769135,27657,0,0,0,5,0,...,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,0.0,0.0,0.0,0.0,0.0,0.0
27658,317103,2769135,317203,2769235,27658,0,0,0,2,0,...,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,0.0,0.0,0.0,0.0,0.0,0.0
27659,317103,2769235,317203,2769335,27659,0,0,0,2,0,...,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
pnt_merge = pd.merge(pnt, nt_edu, on=['x1','x2','y1','y2'] , how='left')
pnt_merge = pd.merge(pnt_merge, nt_hospital, on=['x1','x2','y1','y2','index'], how='left')
pnt_merge = pd.merge(pnt_merge, nt_trans, on=['x1','x2','y1','y2','index'], how='left')
pnt_merge = pd.merge(pnt_merge, nt_other, on=['x1','x2','y1','y2','index'], how='left')
pnt_merge = pd.merge(pnt_merge, nt_store, on=['x1','x2','y1','y2'], how='left')
pnt_merge = pd.merge(pnt_merge, nt_population, on=['x1','x2','y1','y2'], how='left')
pnt_merge = pd.merge(pnt_merge, house, on=['x1','x2','y1','y2'], how='left')
pnt_merge

,x1,y1,x2,y2,index,edu1_100_cnt,edu1_500_cnt,edu1_1000_cnt,edu1_3000_cnt,edu2_100_cnt,...,平均每坪單價_2022,平均每坪單價_2021,平均每坪單價_2020,平均每坪單價_2019,交易總價_2024,交易總價_2023,交易總價_2022,交易總價_2021,交易總價_2020,交易總價_2019
0,278508,2777870,278608,2777970,0,0,0,0,2,0,...,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,0.0,0.0,0.0,0.0,0.0,0.0
1,278508,2777970,278608,2778070,1,0,0,0,2,0,...,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,0.0,0.0,0.0,0.0,0.0,0.0
2,278508,2778070,278608,2778170,2,0,0,0,2,0,...,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,0.0,0.0,0.0,0.0,0.0,0.0
3,278508,2778170,278608,2778270,3,0,0,0,2,0,...,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,0.0,0.0,0.0,0.0,0.0,0.0
4,278508,2778370,278608,2778470,4,0,0,0,2,0,...,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209743,351508,2767770,351608,2767870,209743,0,0,0,3,0,...,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,0.0,0.0,0.0,0.0,0.0,0.0
209744,351608,2766970,351708,2767070,209744,0,0,0,3,0,...,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,0.0,0.0,0.0,0.0,0.0,0.0
209745,351608,2767070,351708,2767170,209745,0,0,0,3,0,...,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,0.0,0.0,0.0,0.0,0.0,0.0
209746,351608,2767270,351708,2767370,209746,0,0,0,3,0,...,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# 把 unnamed:5 改為 district2
pt_merge = pt_merge.rename(columns={'district6':'district1'})
pt_merge = pt_merge.rename(columns={'Unnamed: 5':'district2'})
pt_merge = pt_merge.rename(columns={'Unnamed: 6':'district3'})

pnt_merge = pnt_merge.rename(columns={'district6':'district1'})
pnt_merge = pnt_merge.rename(columns={'Unnamed: 5':'district2'})
pnt_merge = pnt_merge.rename(columns={'Unnamed: 6':'district3'})

# 把平均每坪單價（萬元／坪）空值補0
pt_merge['平均每坪單價（萬元／坪）'] = pt_merge['平均每坪單價（萬元／坪）'].fillna(0)
pnt_merge['平均每坪單價（萬元／坪）'] = pnt_merge['平均每坪單價（萬元／坪）'].fillna(0)

pt_merge


,x1,y1,x2,y2,index,edu1_100_cnt,edu1_500_cnt,edu1_1000_cnt,edu1_3000_cnt,edu2_100_cnt,...,平均每坪單價_2022,平均每坪單價_2021,平均每坪單價_2020,平均每坪單價_2019,交易總價_2024,交易總價_2023,交易總價_2022,交易總價_2021,交易總價_2020,交易總價_2019
0,296103,2777535,296203,2777635,0,0,0,1,30,0,...,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,0.0,0.0,0.0,0.0,0.0,0.0
1,296103,2777635,296203,2777735,1,0,0,1,26,0,...,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,0.0,0.0,0.0,0.0,0.0,0.0
2,296103,2777735,296203,2777835,2,0,0,0,27,0,...,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,0.0,0.0,0.0,0.0,0.0,0.0
3,296103,2777835,296203,2777935,3,0,0,0,27,0,...,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,0.0,0.0,0.0,0.0,0.0,0.0
4,296103,2777935,296203,2778035,4,0,0,0,27,0,...,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27656,317103,2768935,317203,2769035,27656,0,0,0,5,0,...,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,0.0,0.0,0.0,0.0,0.0,0.0
27657,317103,2769035,317203,2769135,27657,0,0,0,5,0,...,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,0.0,0.0,0.0,0.0,0.0,0.0
27658,317103,2769135,317203,2769235,27658,0,0,0,2,0,...,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,0.0,0.0,0.0,0.0,0.0,0.0
27659,317103,2769235,317203,2769335,27659,0,0,0,2,0,...,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# concat
all_merge = pd.concat([pt_merge, pnt_merge], axis=0)

# drop index
all_merge = all_merge.drop(columns=['index'])

all_merge

,x1,y1,x2,y2,edu1_100_cnt,edu1_500_cnt,edu1_1000_cnt,edu1_3000_cnt,edu2_100_cnt,edu2_1000_cnt,...,平均每坪單價_2021,平均每坪單價_2020,平均每坪單價_2019,交易總價_2024,交易總價_2023,交易總價_2022,交易總價_2021,交易總價_2020,交易總價_2019,district1
0,296103,2777535,296203,2777635,0,0,1,30,0,11,...,#DIV/0!,#DIV/0!,#DIV/0!,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,296103,2777635,296203,2777735,0,0,1,26,0,9,...,#DIV/0!,#DIV/0!,#DIV/0!,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,296103,2777735,296203,2777835,0,0,0,27,0,8,...,#DIV/0!,#DIV/0!,#DIV/0!,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,296103,2777835,296203,2777935,0,0,0,27,0,8,...,#DIV/0!,#DIV/0!,#DIV/0!,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,296103,2777935,296203,2778035,0,0,0,27,0,7,...,#DIV/0!,#DIV/0!,#DIV/0!,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209743,351508,2767770,351608,2767870,0,0,0,3,0,0,...,#DIV/0!,#DIV/0!,#DIV/0!,0.0,0.0,0.0,0.0,0.0,0.0,貢寮區
209744,351608,2766970,351708,2767070,0,0,0,3,0,0,...,#DIV/0!,#DIV/0!,#DIV/0!,0.0,0.0,0.0,0.0,0.0,0.0,貢寮區
209745,351608,2767070,351708,2767170,0,0,0,3,0,0,...,#DIV/0!,#DIV/0!,#DIV/0!,0.0,0.0,0.0,0.0,0.0,0.0,貢寮區
209746,351608,2767270,351708,2767370,0,0,0,3,0,0,...,#DIV/0!,#DIV/0!,#DIV/0!,0.0,0.0,0.0,0.0,0.0,0.0,貢寮區


In [10]:
# 寫入 csv
all_merge.to_csv("all_merge.csv", index=False)
pt_merge.to_csv("taipei_merge.csv", index=False) 
pnt_merge.to_csv("new_taipei_merge.csv", index=False) 

In [11]:
all_merge.columns

Index(['x1', 'y1', 'x2', 'y2', 'edu1_100_cnt', 'edu1_500_cnt', 'edu1_1000_cnt',
       'edu1_3000_cnt', 'edu2_100_cnt', 'edu2_1000_cnt', 'edu2_2000_cnt',
       'edu3_100_cnt', 'edu3_1000_cnt', 'edu3_2000_cnt', 'edu_score',
       'hos1_100_cnt', 'hos1_500_cnt', 'hos1_1000_cnt', 'hos1_2000_cnt',
       'hos2_100_cnt', 'hos2_1000_cnt', 'hos2_2000_cnt', 'hos_score',
       'trans1_100_cnt', 'trans1_500_cnt', 'trans1_800_cnt', 'trans2_100_cnt',
       'trans2_500_cnt', 'trans2_1000_cnt', 'trans_score', 'park_100_cnt',
       'park_500_cnt', 'industry_100_cnt', 'industry_1000_cnt',
       'industry_2000_cnt', 'bad_100_cnt', 'bad_500_cnt', 'bad_1000_cnt',
       'bad_2000_cnt', 'park_score', 'industry_score', 'bad_score', '台灣房屋',
       '台灣房屋個數', '信義房屋', '信義房屋個數', '永慶房屋', '永慶房屋個數', '有巢氏房屋', '有巢氏房屋個數',
       '住商房屋', '住商房屋個數', '房仲業個數', 'district', 'district2', 'district3', '人口密度',
       '年齡比例：壯年', '每人平均可支配所得', '方格編號', '房源數', '平均每坪單價（萬元／坪）', '平均每坪單價（萬元／坪）.1',
       '交易總價', '房源數量_2024', '房源數